### TODO:: 

1. build date function and convert utc to mm dd - done!!!

    1.1 - instead of using date select miliseconds as test data from 11.2018 / check if at least 15% of data - done!!!


2. remove duplicate items per single person / remove by bidid  - done my michael - done!!!


3. remove trash columns from android_bids

    3.1 remove marketplace col - done !!!
    
    3.2 device_maker and device_model NaN's changed to unknown and claasified as new device_maker or device_model - done!!!
    
    3.3 
    
    

4. remove trash columns from play_apps

    4.1 Create dataframe from files - done!!!
    
    4.2 Do we need to use play_apps -> Which cols do we want to use??
    
        [video, category, score, histogram, reviews, editors_choice, (price, free), iap, size, installs, content_rating, iap_range, interactive_elements?, bids]


5. trun into comma seperated label for play_apps -> category


6. Merge data from two tables android_bids_us and play_apps

    6.1 USE pandas merge datafreames function to merge both files

In [ ]:
#imports
import pandas as pd
import numpy as np
from datetime import datetime
from zipfile import ZipFile
import pickle
from pathlib import Path


from sklearn.preprocessing import LabelEncoder, StandardScaler


In [ ]:
#Read orig android data
origdata = pd.read_csv('data/android_bids_us.csv')
origdata.head()


# Analyze android dataset

In [ ]:
dataTypes = origdata.dtypes
print(f'Total columns: {dataTypes.size} ||| Total rows: {origdata.count()}')
print(f'Data type of each column of Dataframe : ')
print(dataTypes)
# We may have NAN's in user_isp, device_maker, device_model

print('-----------------------------------')
print('Amount of NaN under utc_time', origdata['utc_time'].isnull().sum())
# all items in marketplace col equal 'chartboost'. we can remove col

#Print start time 
start_time = origdata['utc_time'].min()
end_time = origdata['utc_time'].max()
print('Start Time: ',  datetime.fromtimestamp(start_time/1000))
print('End Time: ',  datetime.fromtimestamp(end_time/1000))

print('-----------------------------------')

print('-----------------------------------')
print('Variance data of android dataset')
print(origdata.var(ddof=0))
print('-----------------------------------')


print('-----------------------------------')
print(origdata.marketplace.unique())
# all items in marketplace col equal 'chartboost'. we can remove col
print('-----------------------------------')

print('-----------------------------------')
print('app_id::')
print('Amount of NaN under app_id', origdata['app_id'].isnull().sum())
print(origdata.app_id.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('user_isp::')
print('Amount of NaN under user_isp', origdata['user_isp'].isnull().sum())
print(origdata.user_isp.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('user_state::')
print(origdata.user_state.value_counts())
#Consider use onehotencoding insted of laelencoding
print('-----------------------------------')

print('-----------------------------------')
print('device_maker::')
print(origdata.device_maker.value_counts())
print('-----------------------------------')

print('-----------------------------------')
print('device_model::')
print(origdata.device_model.value_counts())
print('-----------------------------------')



# Prepare android dataset

In [ ]:
#Rearrange dataframe
#clone orig dataframe
android_df = origdata.copy()





#-----------------------------

#Add date from utc and remove utc
#android_df['date'] = android_df['utc_time'].apply(lambda x: datetime.fromtimestamp(x/1000))
#android_df = android_df.drop(['utc_time'], axis=1)

#-----------------------------

#Remove marketplace col
android_df = android_df.drop(['marketplace'], axis=1)

#-----------------------------

#Remove duplicated bidid. Keep only single instance keep the one with click == 1
android_df = android_df[~((android_df['bidid'].duplicated(keep=False))&(android_df['click']==0))]
#and then remove bids column
android_df = android_df.drop(['bidid'], axis=1)

#-----------------------------

#-----------------------------
# user_isp
android_df['user_isp'] = android_df['user_isp'].fillna(method="ffill")
#-----------------------------


#-----------------------------
#Change device_maker NaN's to unknown manufacturerer
android_df['device_maker'] = android_df['device_maker'].apply(lambda x: 'unknown' if x!=x else x)


#Change device_model NaN's to unknown valid model
android_df['device_model'] = android_df['device_model'].apply(lambda x: 'unknown' if x!=x else x)
#-----------------------------


#-----------------------------
# Take size cols and transform into resolution
android_df['screen_resolution'] = android_df['device_height']*android_df['device_width']
android_df.drop(['device_width', 'device_height'], axis=1, inplace=True)

# And then do scaling to new col
scaler = StandardScaler()
android_df['screen_resolution'] = scaler.fit_transform(android_df[['screen_resolution']])
#-----------------------------


#-----------------------------
# LabelEncoder for user_state

#Consider use onehotencoding insted of laele ncoding

le1 = LabelEncoder()
android_df['user_state'] = le1.fit_transform(android_df['user_state'])
android_df
#-----------------------------

#-----------------------------
# LabelEncoder for user_isp

#Consider use onehotencoding insted of laele ncoding

le2 = LabelEncoder()
android_df['user_isp'] = le2.fit_transform(android_df['user_isp'])
android_df
#-----------------------------





android_df

# Extract play_apps data and first clean

In [ ]:
# Get play_apps data


#Read orig play_apps data
#app_details_orig = ZipFile("data/play_apps.zip")
#app_file = 'play_apps/a008.com.fc2.blog.androidkaihatu.datecamera2'
#app = pickle.loads(app_details_orig.read(app_file))
#app


def zipToDataFrame(zipped, to_remove):
    rows = []
    
    for f in zipped.filelist:
        app_file = f.filename
        try:
            app = pickle.loads(zipped.read(app_file))
        except EOFError as e:
            continue
        except pickle.UnpicklingError as e:
            continue
        except Exception as e:
            return
            
        # Remove columns
        for col in to_remove:
            del app[col]
        rows.append(app)
    
    return pd.DataFrame(rows)


app_details = ZipFile("data/play_apps.zip")
cols_to_remove = ['title','screenshots', 'icon', 'size',
                  'histogram', 'description', 'description_html', 
                  'recent_changes', 'developer', 'developer_id', 'developer_email', 'developer_url',
                  'developer_address', 'url', 'current_version', 'updated', 'required_android_version']


orig_play_apps = zipToDataFrame(app_details, cols_to_remove)
orig_play_apps



# Analyze play_apps dataset

In [ ]:
# explore play_apps data characteristics

apps_dataTypes = orig_play_apps.dtypes

print(f'Total columns: {apps_dataTypes.size} ||| Total rows: {orig_play_apps.count()}')
print(f'Data type of each column of Dataframe : ')
print(apps_dataTypes)
# We may have NAN's in iap_range, interactive_elements, price

print('-----------------------------------')
print('Amount of NaN under price', orig_play_apps['price'].isnull().sum())
print('Amount of NaN under free', orig_play_apps['free'].isnull().sum())
print('price values: ', orig_play_apps.price.unique())
print('free values: ',orig_play_apps.free.unique())

print('How many free from all dataset: ', (orig_play_apps['free'] == True).sum())
# items in price and free should be in a single col
# we can take the free col since the non free item is very little : about 1000 out of 29000
# or we can take col price since 0 represnts free
print('-----------------------------------')

print('-----------------------------------')
print('Variance data of play aps dataset')
print(orig_play_apps.var(ddof=0))
print('-----------------------------------')

print('-----------------------------------')
# items in price and free should be in a single col
print('Amount of NaN under score', orig_play_apps['score'].isnull().sum())
print('score: ', orig_play_apps.score.unique())
print('-----------------------------------')


print('-----------------------------------')
# Test reviews col - need to be scaled
print('Amount of NaN under reviews', orig_play_apps['reviews'].isnull().sum())
print('reviews: ', orig_play_apps.reviews.unique())
print('-----------------------------------')

print('-----------------------------------')
# Test bids col - need to be scaled
print('Amount of NaN under bids', orig_play_apps['bids'].isnull().sum())
print('bids: ', orig_play_apps.bids.unique())
print('-----------------------------------')


print('-----------------------------------')
print('content_rating column::: ')
# Test content_rating col
rating = orig_play_apps['content_rating'].agg(lambda x: ':'.join(map(str, x)))
print(rating.unique())
# We might drop this column since its require a lot of handling and im not sure its gonna impact our model
print('-----------------------------------')


print('-----------------------------------')
print('category column::: ')
# Test category col
category = orig_play_apps['category'].agg(lambda x: ':'.join(map(str, x)))
print(category.unique())
# We might drop this column since its require a lot of handling and im not sure its gonna impact our model
print('-----------------------------------')

print('-----------------------------------')
print('interactive_elements column::: ')
# Test category col
print('Amount of NaN under interactive_elements', orig_play_apps['interactive_elements'].isnull().sum())
interactive = orig_play_apps['interactive_elements'].agg(lambda x: ':'.join(map(str, x)))
interactive
#print(interactive.unique())
# We might drop this column since its require a lot of handling and im not sure its gonna impact our model
print('-----------------------------------')





# Prepare play_apps dataset

In [ ]:
#Fix and manipulate play_apps data before merge with android_bids_us
#clone orig dataframe
play_apps_df = orig_play_apps.copy()

# Convert String to int.
def convert_installs(str_val=''):
    str_val = str_val.translate({ord(i):None for i in ',+'})
    return int(str_val)

# Convert String to float.
def convert_price(str_val=''):
    str_val = str_val.translate({ord(i):None for i in '$'})
    return float(str_val)


#---------------------------------------------------
# Handle 'Installs' column
# Translate to number and then scale it
play_apps_df['installs'].fillna(method='pad', inplace=True)
play_apps_df['installs'] = play_apps_df['installs'].apply(lambda s: convert_installs(s))

# And then do scaling to new col
scaler = StandardScaler()
play_apps_df['installs'] = scaler.fit_transform(play_apps_df[['installs']])
#---------------------------------------------------


#---------------------------------------------------
# Handle 'video' column
play_apps_df['video'] = play_apps_df['video'].apply(lambda x: 0 if pd.isna(x) else 1)
#---------------------------------------------------

#---------------------------------------------------
# Handle 'score ' column
# Translate to number
play_apps_df['score'] = pd.to_numeric(play_apps_df['score'],errors = 'coerce')
play_apps_df['score'].fillna(method='pad', inplace=True)
play_apps_df['score'] = scaler.fit_transform(play_apps_df[['score']])
#---------------------------------------------------

#---------------------------------------------------
# Handle 'reviews' column - we cant know weither the review is good or bad, 
# but the amount of reviews can point on popularity off the app
# There are no NaN's
# Translate to number
scaler = StandardScaler()
play_apps_df['reviews'] = scaler.fit_transform(play_apps_df[['reviews']])
#---------------------------------------------------


#---------------------------------------------------
#price and free can be merged into single col named price since 0 represntes free app
# and scale it
play_apps_df['price'] = play_apps_df['price'].fillna(method='pad', inplace=False)
play_apps_df['price'] = play_apps_df['price'].apply(lambda s: convert_price(s))
play_apps_df['price'] = scaler.fit_transform(play_apps_df[['price']])
play_apps_df.drop(['free'], axis=1, inplace=True)
#---------------------------------------------------



play_apps_df



# Merge two datasets

In [ ]:
print('---------------------------------------------')
#merge two data sets into merged dataframe
merged_df = pd.merge(android_df, play_apps_df, on='app_id')
merged_df = merged_df.drop(['app_id'], axis=1)

print('Rows before merge: ',android_df.shape[0])
print('Rows after merge: ', merged_df.shape[0])

#Concolusion after merge
# from android = 2936921 rows × 10 columns turn into 2656577 rows × 24 columns
# we missed ~300k rows. need to check y.....
print('---------------------------------------------')


merged_df




# Split data and save to files

In [ ]:
print('---------------------------------------------')
print('Amount of NaN under utc_time: ', merged_df['utc_time'].isnull().sum())
print('Amount of NaN in all data : ', merged_df.isnull().sum().sum())
print('---------------------------------------------')


print('---------------------------------------------')
#Print start time after merge
merged_start_time = merged_df['utc_time'].min()
merged_end_time = merged_df['utc_time'].max()
print('Start Time: ',  datetime.fromtimestamp(merged_start_time/1000))
print('End Time: ',  datetime.fromtimestamp(merged_end_time/1000))
print('Split time: ',  datetime.fromtimestamp(1541023200000/1000))
print('---------------------------------------------')


print('---------------------------------------------')
tarin_df, test_df = merged_df[(mask:=merged_df['utc_time'] < 1541023200000)], merged_df[~mask]

tarin_df = tarin_df.drop(['utc_time'], axis=1)
test_df = test_df.drop(['utc_time'], axis=1)

tarin_size = tarin_df.shape[0]
test_size = test_df.shape[0]

print('Amount of tarin_df data : ', tarin_size)
print('Amount of test_df data : ', test_size)

print(f'Test data is: {(test_size / (tarin_size + test_size))*100} percent')

      
print('---------------------------------------------')
      
#Save to CSV files
#Save to splited csv files
filepath1 = Path('data/train.csv')  
filepath2 = Path('data/test.csv')  
tarin_df.to_csv(filepath1, index=False)
test_df.to_csv(filepath2, index=False)

